<h3>Produced By: Garrett Zimmerman</h3>
<h3>Version 1: January 4th, 2024</h3>
<h2>Goal:</h2> 
<p>Automate Youtube Upload process. I would like a python script that will grab a video file and upload it to my account. Make sure your code is optimized for performance. I would like to see a basic demo of the script upload youtube videos to your personal account. This is going to be very important for one of our products and our business operations. We will extend the framework to any social media platform</p>

<h2>Action Plan:</h2>
<ol>
    <li>Grab a video/videos file from a folder</li>
    <li>Upload the video file to youtube account</li>
    <li>Optimize the process</li>
</ol>

<h1>Actions Items Needed to Complete before continuing:</h1>
<dl>
    <dt>1. Google Developer Console Account:</dt> 
        <dd>A:  You need a Google account to access the Google Developer Console. (COST $25 for life)</dd>
    <dt>2. YouTube Data API Key: </dt>
        <dd>You'll need to create a project in the Google Developer Console and enable the YouTube Data API for that project.</dd>
        <dd>A:  Sign in to the Google Developer Console</dd>
        <dd>B:  Go to the Google Developer Console and sign in with your Google account.</dd>
        <dd>C:  Once you're logged in, create a new project by clicking on "Select a project" or "New Project" at the top of the page.</dd>
        <dd>D:  Give your project a name and create it.</dd>
        <dd>E:  Enable the YouTube Data API for Your Project</dd>
        <dd>F:  With the project selected, navigate to the "Library" in the Google Developer Console.</dd>
        <dd>G:  Search for "YouTube Data API v3" and select it.</dd>
        <dd>H:  Click on "Enable" to add the YouTube Data API to your project. This step is crucial as it allows your application to interact with YouTube.</dd>
    <dt>3. OAuth 2.0 Client ID: For uploading videos, you need to authenticate via OAuth 2.0. In the Google Developer Console, create OAuth 2.0 credentials.</dt>
        <dd>A:  Configure the OAuth Consent Screen:</dd>
        <dd>B:  In the Developer Console, go to the "OAuth consent screen" section.</dd>
        <dd>C:  Select the user type (usually "External" for apps that are available to any user with a Google account).</dd>
        <dd>D:  Fill in the required fields like the app name, user support email, and developer contact information. This information will be shown to users when they are asked to grant permissions to your app.</dd>
        <dd>E:  Create OAuth 2.0 Credentials:</dd>
        <dd>F:  Go to the "Credentials" tab in the Developer Console.</dd>
        <dd>G:  Click on "Create Credentials" and choose "OAuth client ID".</dd>
        <dd>H:  You might need to configure the consent screen before you can create credentials.</dd>
        <dd>I:  Add authorized redirect URIs if needed (this is essential for web applications).</d>
        <dd>J:  Obtain the Client ID and Client Secret:</dd>
        <dd>K:  Once you create the OAuth client ID, you'll receive a client ID and client secret.</dd> 
        <dd>L:  These are important for your Python script to authenticate with Google's servers.</dd>
        <dd>M:  Download the JSON Configuration File:</dd>
        <dd>N:  You can download the configuration file that contains your client ID and client secret. This file is often named client_secrets.json.</dd>
        <dd>O:  This file will be used in your Python script to facilitate OAuth 2.0 authentication</dd>
</dl>

In [18]:
#All modules needed
import os
from typing import List
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import googleapiclient.http
import pandas as pd
import pkg_resources



<h2>The function find_video_files will take in a folder path and return a the list of videos in that file by looking for .mp4</h2>
<p>Make sure and change the file location if need be</p>

In [ ]:
def find_video_files(folder_path: str, file_extension: str = ".mp4") -> List[str]:
    """
    Finds video files in the specified folder with the given file extension.

    :param folder_path: Path to the folder where video files are located.
    :param file_extension: The extension of the video files to find.
    :return: List of video file paths.
    """
    try:
        video_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(file_extension)]
        assert len(video_files) > 0, "No video files found in the folder."
        return video_files
    except Exception as e:
        print(f"Error finding video files: {e}")
        return []


<h1>Every Video Updated to Youtube will need to have this information:</h1>
<ul>
    <li>title </li> 
    <li>description</li> 
    <li>category </li> 
    <li>tags </li> 
    <li>privacy status </li> 
</ul>

<h1>Two Categories require specified responses</h1>
<h2>Privacy status:</h2>
<dl>
    <dt>'public'</dt> 
        <dd>The video can be viewed by anyone </dd>
    <dt>'private'</dt> 
        <dd>The video can only be viewed by the uploader and the users specifically granted permission</dd>
    <dt>'unlisted'</dt> 
        <dd>The video will not appear in any of YouTube's public spaces, but anyone with the link can view it</dd>
</dl>
<h2>Category</h2>
<p>Category Must be choosen from a list of numbers representing the catergory. Run the cell below to see a table of avaliable categories.</p>  

In [2]:
# Data for YouTube category IDs
category_data = {
    "Category ID": [
        "1", "2", "10", "15", "17", "18", "19", "20", 
        "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"
    ],
    "Category Name": [
        "Film & Animation", "Autos & Vehicles", "Music", "Pets & Animals",
        "Sports", "Short Movies", "Travel & Events", "Gaming",
        "Videoblogging", "People & Blogs", "Comedy", "Entertainment",
        "News & Politics", "Howto & Style", "Education", "Science & Technology",
        "Nonprofits & Activism", "Movies"
    ]
}

# Create a DataFrame
df = pd.DataFrame(category_data)

# Display the DataFrame
df

,Category ID,Category Name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [ ]:
#This was the first function created. There are some speed improvements below:

def upload_video(file_path):
    # User inputs for video details
    title = input(f"Enter the title for the video '{os.path.basename(file_path)}': ")
    description = input("Enter the description for the video: ")
    category = input("Enter the category ID for the video (e.g., '22' for People & Blogs): ")
    tags = input("Enter the tags for the video (comma-separated): ").split(',')
    privacy_status = input("Enter the privacy status (public, private, or unlisted): ")

    assert privacy_status in ['public', 'private', 'unlisted'], "Invalid privacy status."

    # OAuth 2.0 setup
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for local testing!
    client_secrets_file = "client_secrets.json"

    scopes = ["https://www.googleapis.com/auth/youtube.upload"]
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes=scopes)

    # Using run_local_server
    credentials = flow.run_local_server()

    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)


    request_body = {
        "snippet": {
            "categoryId": category,
            "title": title,
            "description": description,
            "tags": tags
        },
        "status": {
            "privacyStatus": privacy_status
        }
    }

    media_file = googleapiclient.http.MediaFileUpload(file_path)

    # Call the API to upload the video
    request = youtube.videos().insert(
        part="snippet,status",
        body=request_body,
        media_body=media_file
    )

    response = request.execute()
    print(f"Upload successful for {file_path}.")

<h2>Uploading Videos From file:</h2>

In [ ]:

#Enter the folder where the videos are
folder_path = "D:\Test_videos"  # Replace with your actual folder path
video_files = find_video_files(folder_path)

print(video_files)  # This will print the list of found video files
for video_file in video_files:
    upload_video(video_file)
    print("Video Uploaded:",video_file)

<h2>Creating a requirements txt</h2>

In [ ]:
packages = ["google-auth-oauthlib", "google-api-python-client", "pandas"]
with open("requirements.txt", "w") as f:
    for package in packages:
        version = pkg_resources.get_distribution(package).version
        f.write(f"{package}=={version}\n")

<h2>Speed Improvements</h2>
<h3>Separate OAuth Initialization from Upload Function</h3>
<p> Initialize the OAuth flow and YouTube client once, outside the upload_video function. This prevents re-authenticating and rebuilding the client for each video upload, which is time-consuming and inefficient.</p>

In [16]:
def initialize_youtube_client():
    try:
        os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # For local testing only!
        client_secrets_file = "client_secrets.json"
        scopes = ["https://www.googleapis.com/auth/youtube.upload"]
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
            client_secrets_file, scopes=scopes)
        credentials = flow.run_local_server()
        return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)
    except Exception as e:
        print(f"Error initializing YouTube client: {e}")
        return None

In [22]:
def upload_video(youtube, file_path, title, description, category, tags, privacy_status):
    try:
        request_body = {
            "snippet": {
                "categoryId": category,
                "title": title,
                "description": description,
                "tags": tags
            },
            "status": {
                "privacyStatus": privacy_status
            }
        }

        media_file = googleapiclient.http.MediaFileUpload(file_path)

        request = youtube.videos().insert(
            part="snippet,status",
            body=request_body,
            media_body=media_file
        )

        response = request.execute()
        print(f"Upload successful for {file_path}. Response:", response)
    except googleapiclient.errors.HttpError as e:
        print(f"An HTTP error occurred: {e.resp.status} {e.content}")
    except Exception as e:
        print(f"An error occurred during video upload: {e}")

In [ ]:
def main():
    folder_path = "D:\\Test_videos"  # Use double backslashes for Windows paths
    video_files = find_video_files(folder_path)

    youtube = initialize_youtube_client()
    if youtube is None:
        return

    for video_file in video_files:
        title = input(f"Enter the title for the video '{os.path.basename(video_file)}': ")
        description = input("Enter the description for the video: ")
        category = input("Enter the category ID for the video: ")
        tags = input("Enter the tags for the video (comma-separated): ").split(',')
        privacy_status = input("Enter the privacy status (public, private, or unlisted): ")

        upload_video(youtube, video_file, title, description, category, tags, privacy_status)
        print("Video Uploaded:", video_file)

if __name__ == "__main__":
    main()

<h2>Further Improvemnts and Considerations</h2>
<dl>
    <dt>User Input</dt> 
    <dd>Currently, the script still asks for input for each video. Depending on our use case, we might automate this or use a different approach to handle metadata.<dd>
</dl>

<h2>Createing an Excel Spreadsheet to improve perfomance and to release on a set schedule<h2>
<p>My initial idea is to have the program look at the spread sheet typ and then be able to dtermine if we are uploading a comment or a video.<p>
<dl>
    <dt>1. Create Excel Spread Sheet</dt>
    <dd>User will have to uploade the video title, and choose from several options before running the program. The sheet will have all information stored need for youtube video or comments in the sheet. The only exception will be the hashtags we will use api to get bettter more relavent ones</dd>
    <dt>2. Upload Excel sheet into python and then search for relevant hashtags</dt>
    <dd>Must be in form: #hashtag1,#hashtag2,...,#hashtagN</dd>
    <dd>pip install pandas openpyxl</dd>
    <dt>3. Get Time Stamps from excel sheet and create an upload schedule</dt>
    <dt>4. Upload video at set time</dt>
</dl>

In [26]:
import pandas as pd

def read_first_sheet(file_path):
    """
    Reads the first sheet of an Excel file and returns all its columns.

    :param file_path: Path to the Excel file.
    :return: A DataFrame containing all columns from the first sheet.
    """
    # Read the first sheet of the Excel file
    df = pd.read_excel(file_path, engine='openpyxl', sheet_name=0)
    
    return df

In [27]:
my_df= read_first_sheet("D:\Automated Youtube Upload\Automated_youtube_upload_sheet.xlsx")
my_df

d:\Automated Youtube Upload\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Video Title,Video File Name,Description,Privacy Status,Video Category,Video Category Number,Release Date,Time (Format 00:00 AM/PM),Convert to Military time,Time Zone,UTC Time
0,Testing Video,GZIMM_test_1.mp4,Testing youtube upload,private,27 - Education,27,2024-02-27,08:00:00,08:00:00,MST-Moutain Standard Time,15:00:00
1,Testing Video 2,GZIMM_test_2.mp4,Testing a Time release youtube upload,private,27 - Education,27,2024-02-27,10:05:00,10:05:00,EST-Eastern Standard Time,15:05:00


Updateing the upload video retireval funcion to look at eh given 

In [7]:
import os
from typing import Optional
def find_specific_video_file(video_file_name: str, folder_path: str = "D:\Test_videos") -> Optional[str]:
    """
    Finds a specific video file by name in the specified folder.

    :param video_file_name: The name of the video file to find.
    :param folder_path: Path to the folder where video files are located. Defaults to "D://Videos/".
    :return: Path to the video file if found, None otherwise.
    """
    try:
        # Ensure the video file name includes the extension, e.g., '.mp4'
        video_files = [f for f in os.listdir(folder_path) if f == video_file_name]

        if len(video_files) == 1:
            # Return the full path to the found video file
            return os.path.join(folder_path, video_files[0])
        elif len(video_files) > 1:
            print(f"Multiple files with the name {video_file_name} found. Returning the first one.")
            return os.path.join(folder_path, video_files[0])
        else:
            print("Video file not found.")
            return None
    except Exception as e:
        print(f"Error finding video file: {e}")
        return None

Creating Relative hashtags from description
pip install python-dotenv

In [10]:
from dotenv import load_dotenv
# Load the environment variables from .env file
load_dotenv()
#API Keys and Envriornments we will need in this notebook.
required_env_vars = {
    "OPENAI_API_KEY": "OPENAI_API_KEY",
}
# Iterate through the required environment variables
for var, name in required_env_vars.items():
    value = os.environ.get(var)
    if value is None:
        raise ValueError(f"{name} is not set in the environment variables")

Turning above into a fucntion

In [29]:
def load_and_verify_env_vars(required_vars):
    """
    Load environment variables from a .env file and verify if the required ones are set.

    :param required_vars: A dictionary where keys are the names of the required environment
                          variables and values are the human-readable names or descriptions.
    """
    load_dotenv()  # Load the environment variables from .env file

    missing_vars = []
    for var, name in required_vars.items():
        if os.environ.get(var) is None:
            missing_vars.append(f"{name} ({var})")

    if missing_vars:
        missing = ', '.join(missing_vars)
        raise ValueError(f"Missing required environment variables: {missing}")

In [14]:
from openai import OpenAI
def create_hashtag(video_description, number_of_hash_tags):
    client = OpenAI()
    prompt = f"Context: {video_description}\nWhat are the most popular hashtags related to the description?\nAnswer:"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are helping someone create relative hashtags to get their videos seen more"},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Assuming the API response is a single string of hashtags separated by spaces and prefixed with '#'
    hashtags = response.choices[0].message.content.strip().split(' ')
    
    # Filtering out empty strings in case of extra spaces and ensuring we only get the desired number of hashtags
    hashtags = [tag for tag in hashtags if tag][0:number_of_hash_tags]
    
    # Joining the hashtags into the desired format: '#hashtag1,#hashtag2,...,#hashtagn'
    formatted_hashtags = ','.join(hashtags)
    
    return formatted_hashtags

In [15]:
video_descritpion='Testing a Time release youtube upload'
num_hashtags=5
my_hash_tags=create_hashtag(video_descritpion, num_hashtags)
my_hash_tags

'#timelapse,#youtube,#upload,#testing,#videorelease'

In [30]:
def main():
    folder_path_video = "D:\\Test_videos"  # Use double backslashes for Windows paths
    folder_path_excel = "D:\\Automated Youtube Upload\\Automated_youtube_upload_sheet.xlsx"
    num_hashtags = 8
    #Getting API Keys From Local Environment 
    required_env_vars = {
        "OPENAI_API_KEY": "OpenAI API Key",
    }
    try:
        load_and_verify_env_vars(required_env_vars)
        # Your main logic here
        print("All required environment variables are set. Proceeding with main logic.")
    except ValueError as e:
        print(f"Error: {e}")
        return  # Exit the main function if environment variables are missing
    
    # Initialize YouTube 
    youtube = initialize_youtube_client()
    if youtube is None:
        return
    # Creating a DataFrame to pull the remaining info for the videos
    my_df = read_first_sheet(folder_path_excel)
    
    for index, row in my_df.iterrows():
        video_file_name = row['Video File Name']
        video_file_path = find_specific_video_file(video_file_name, folder_path_video)
        if not video_file_path:  # If video file is not found, skip to the next iteration
            print(f"Video file {video_file_name} not found in {folder_path_video}.")
            continue
        title = row['Video Title']
        description = row['Description']
        category = str(row['Video Category Number'])  # Ensure the category is a string
        tags = create_hashtag(description, num_hashtags)
        privacy_status = row['Privacy Status']
        
        # Upload video
        upload_video(youtube, video_file_path, title, description, category, tags, privacy_status)
        print("Video Uploaded:", video_file_name)

if __name__ == "__main__":
    main()

All required environment variables are set. Proceeding with main logic.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=528310130399-420qqkvl1id397okjrg829g4nf5t8m8l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=yc6iRgXeOQhlU7bJbt9VLXwCLY7K1W&access_type=offline


d:\Automated Youtube Upload\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Upload successful for D:\Test_videos\GZIMM_test_1.mp4. Response: {'kind': 'youtube#video', 'etag': 'gOkoorIxE-mFriTBsAE_KqdDaEU', 'id': 'm6BdqMWo5IE', 'snippet': {'publishedAt': '2024-02-27T15:55:22Z', 'channelId': 'UCfxqTJlCCG8HUTzM6KSksWA', 'title': 'Testing Video', 'description': 'Testing youtube upload', 'thumbnails': {'default': {'url': 'https://i9.ytimg.com/vi/m6BdqMWo5IE/default.jpg?sqp=CNSL-K4G&rs=AOn4CLBR9qEkpDarM9IS8fR40MRUGh3nbA', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i9.ytimg.com/vi/m6BdqMWo5IE/mqdefault.jpg?sqp=CNSL-K4G&rs=AOn4CLAQq_wI3Rgshfmipvu-x8faurAI6A', 'width': 320, 'height': 180}, 'high': {'url': 'https://i9.ytimg.com/vi/m6BdqMWo5IE/hqdefault.jpg?sqp=CNSL-K4G&rs=AOn4CLDscda1_7CLhd-WluTK65vzBc4Liw', 'width': 480, 'height': 360}}, 'channelTitle': 'ThemanfromCO', 'tags': ['For', 'YouTube', 'related', 'testing', 'to', 'uploads', 'videos', 'you'], 'categoryId': '27', 'liveBroadcastContent': 'none', 'localized': {'title': 'Testing Video', 'description':